# Chapter 04 실습: 데이터 파이프라인 구축

## 목표
CIFAR-10 데이터셋으로 완전한 학습용 파이프라인을 구축하고 시각화한다.

### 실습 구성
| 실습 | 내용 |
|------|------|
| 실습 1 | CIFAR-10 데이터 로드 및 탐색 |
| 실습 2 | 전처리 파이프라인 (정규화 + cache/shuffle/batch/prefetch) |
| 실습 3 | 데이터 증강 레이어 추가 |
| 실습 4 | 시각화 (4×4 그리드, 클래스 레이블) |

### 도전 과제 (선택)
1. 증강 강도를 높여 (rotation=0.3) 시각화해보기
2. Z-score 정규화 방법으로 바꿔보기
3. `prefetch` 없을 때와 있을 때 배치 로드 시간 측정

In [ ]:
import sys
import os

# 상위 디렉토리(chapter04_data_pipeline)를 모듈 경로에 추가
# 공통 유틸리티가 있을 경우 import 가능
sys.path.append('..')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성을 위한 시드 고정
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

AUTOTUNE = tf.data.AUTOTUNE

print('TensorFlow 버전:', tf.__version__)
print('Python 버전:', sys.version)
print('GPU 사용 가능:', tf.config.list_physical_devices('GPU'))

## 실습 1: 데이터 로드

In [ ]:
# -----------------------------------------------------------
# CIFAR-10 데이터셋 로드
# - 60,000장의 32×32 컬러 이미지 (훈련 50,000 + 테스트 10,000)
# - 10개 클래스: airplane, automobile, bird, cat, deer,
#                dog, frog, horse, ship, truck
# -----------------------------------------------------------

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# y는 (N, 1) shape → (N,)으로 squeeze
y_train = y_train.squeeze()
y_test  = y_test.squeeze()

CLASS_NAMES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

print('=== 데이터셋 정보 ===')
print(f'훈련 이미지: {x_train.shape}  dtype: {x_train.dtype}')
print(f'훈련 레이블: {y_train.shape}  dtype: {y_train.dtype}')
print(f'테스트 이미지: {x_test.shape}')
print(f'클래스 수: {len(CLASS_NAMES)}')
print(f'픽셀값 범위: [{x_train.min()}, {x_train.max()}]')

# 클래스별 샘플 수 확인
print('\n=== 클래스별 훈련 샘플 수 ===')
for cls_id, cls_name in enumerate(CLASS_NAMES):
    count = np.sum(y_train == cls_id)
    print(f'  [{cls_id}] {cls_name:12s}: {count}개')

In [ ]:
# -----------------------------------------------------------
# 원본 데이터 시각화: 각 클래스 대표 이미지
# -----------------------------------------------------------

fig, axes = plt.subplots(2, 5, figsize=(14, 6))
fig.suptitle('CIFAR-10 클래스별 대표 이미지 (원본 uint8)', fontsize=14)

for cls_id, ax in enumerate(axes.flat):
    # 해당 클래스의 첫 번째 샘플
    idx = np.where(y_train == cls_id)[0][0]
    ax.imshow(x_train[idx])
    ax.set_title(f'[{cls_id}] {CLASS_NAMES[cls_id]}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 실습 2: 전처리 파이프라인

In [ ]:
# -----------------------------------------------------------
# Min-Max 정규화 함수
#
# 수식: x' = (x - x_min) / (x_max - x_min)
# CIFAR-10의 경우 x_min=0, x_max=255이므로 단순히 / 255.0
#
# 결과: 픽셀값 범위 [0, 255] → [0.0, 1.0]
# -----------------------------------------------------------

def normalize_minmax(image, label):
    """Min-Max 정규화: uint8 [0,255] → float32 [0.0, 1.0]"""
    image = tf.cast(image, tf.float32)
    image = image / 255.0  # x_min=0, x_max=255 고정
    return image, label

# -----------------------------------------------------------
# 파이프라인 구성: cache → shuffle → batch → prefetch
# -----------------------------------------------------------

BATCH_SIZE  = 32
BUFFER_SIZE = 10_000  # shuffle 버퍼 크기

def build_base_pipeline(x, y, batch_size=BATCH_SIZE, training=True):
    """
    기본 전처리 파이프라인 (증강 없음).
    x: numpy uint8 이미지 배열
    y: numpy 레이블 배열
    training: True이면 셔플 포함
    """
    ds = tf.data.Dataset.from_tensor_slices((x, y))

    # 1단계: 개별 샘플 정규화 (병렬)
    ds = ds.map(normalize_minmax, num_parallel_calls=AUTOTUNE)

    # 2단계: 캐시 — 첫 에포크 전처리 결과를 메모리에 저장
    #         두 번째 에포크부터는 캐시에서 읽어 속도 향상
    ds = ds.cache()

    # 3단계: 셔플 (훈련 시에만)
    if training:
        ds = ds.shuffle(buffer_size=BUFFER_SIZE, seed=SEED)

    # 4단계: 배치 생성
    ds = ds.batch(batch_size, drop_remainder=training)

    # 5단계: 프리패치 — GPU 연산 중 CPU가 다음 배치 준비
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds_base = build_base_pipeline(x_train, y_train, training=True)
test_ds_base  = build_base_pipeline(x_test,  y_test,  training=False)

print('=== 기본 파이프라인 구성 완료 ===')
print('훈련 Dataset spec:', train_ds_base.element_spec)

# 정규화 결과 확인
for imgs, lbls in train_ds_base.take(1):
    print(f'\n배치 shape: {imgs.shape}')
    print(f'픽셀값 범위: [{imgs.numpy().min():.4f}, {imgs.numpy().max():.4f}]')
    print(f'평균: {imgs.numpy().mean():.4f}, 표준편차: {imgs.numpy().std():.4f}')

## 실습 3: 데이터 증강 추가

In [ ]:
# -----------------------------------------------------------
# Keras Sequential 증강 레이어
# 파이프라인의 .map() 단계에 삽입하여 CPU에서 처리한다
# -----------------------------------------------------------

augmentation = tf.keras.Sequential([
    # 수평 뒤집기: 50% 확률 (자동차, 배 등에 적합)
    tf.keras.layers.RandomFlip('horizontal', seed=SEED),

    # ±15% 회전 (0.15 ≈ 8.6°)
    # 너무 크면 숫자/텍스트 분류에서 오히려 해로울 수 있다
    tf.keras.layers.RandomRotation(factor=0.15, seed=SEED),

    # ±10% 크기 조정
    tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1, seed=SEED),
], name='augmentation')

def augment_fn(image, label):
    """파이프라인용 증강 래퍼 함수."""
    # training=True: 훈련 시에만 랜덤 증강 활성화
    image = augmentation(image, training=True)
    return image, label

# -----------------------------------------------------------
# 증강 포함 파이프라인 구성
# 증강은 배치 이후에 적용: 배치 단위로 한 번에 처리
# -----------------------------------------------------------

def build_augmented_pipeline(x, y, batch_size=BATCH_SIZE, training=True):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(normalize_minmax, num_parallel_calls=AUTOTUNE)
    ds = ds.cache()
    if training:
        ds = ds.shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
    ds = ds.batch(batch_size, drop_remainder=training)
    if training:
        # 배치 단위 증강: num_parallel_calls로 CPU 병렬 처리
        ds = ds.map(augment_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds_aug = build_augmented_pipeline(x_train, y_train, training=True)
test_ds      = build_augmented_pipeline(x_test,  y_test,  training=False)

print('=== 증강 파이프라인 구성 완료 ===')
print('훈련 Dataset spec:', train_ds_aug.element_spec)
print('증강 레이어 구성:')
augmentation.summary()

## 실습 4: 시각화

In [ ]:
# -----------------------------------------------------------
# 증강된 샘플 16개를 4×4 그리드로 시각화
# 각 셀에 클래스 레이블(한글) 표시
# -----------------------------------------------------------

CLASS_NAMES_KO = [
    '비행기', '자동차', '새', '고양이', '사슴',
    '개', '개구리', '말', '선박', '트럭'
]

# 첫 번째 배치에서 16개 추출
for imgs, lbls in train_ds_aug.take(1):
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    fig.suptitle('증강된 CIFAR-10 샘플 (4×4 그리드)', fontsize=14, y=1.02)

    for i, ax in enumerate(axes.flat):
        # 픽셀값을 [0,1] 범위로 클리핑 후 시각화
        img = tf.clip_by_value(imgs[i], 0.0, 1.0).numpy()
        lbl = lbls[i].numpy()

        ax.imshow(img)
        ax.set_title(
            f'{CLASS_NAMES_KO[lbl]}\n({CLASS_NAMES[lbl]})',
            fontsize=9,
            pad=3
        )
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# -----------------------------------------------------------
# 원본 vs 증강 비교: 동일 이미지 1장을 여러 번 증강
# -----------------------------------------------------------

# 원본 이미지 한 장 선택 (float32 정규화)
sample_img = tf.cast(x_train[0], tf.float32) / 255.0  # (32, 32, 3)
sample_lbl = y_train[0]
sample_batch = tf.expand_dims(sample_img, 0)           # (1, 32, 32, 3)

fig, axes = plt.subplots(2, 5, figsize=(14, 6))
fig.suptitle(
    f'원본: {CLASS_NAMES_KO[sample_lbl]} ({CLASS_NAMES[sample_lbl]}) '
    f'→ 동일 이미지 9번 증강',
    fontsize=13
)

for i, ax in enumerate(axes.flat):
    if i == 0:
        ax.imshow(sample_img.numpy())
        ax.set_title('원본', fontsize=10, color='red')
    else:
        # training=True: 랜덤 증강 적용
        aug_img = augmentation(sample_batch, training=True)[0]
        ax.imshow(tf.clip_by_value(aug_img, 0.0, 1.0).numpy())
        ax.set_title(f'증강 #{i}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 도전 과제

### 도전 1: 증강 강도 높이기 (rotation=0.3)

아래 셀에서 `RandomRotation(factor=0.3)`으로 변경하고 시각화를 다시 실행해보자.

In [ ]:
# 도전 1: 더 강한 증강 레이어
augmentation_strong = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', seed=SEED),
    tf.keras.layers.RandomRotation(factor=0.3, seed=SEED),   # 0.15 → 0.3으로 강화
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2, seed=SEED),
    tf.keras.layers.RandomContrast(factor=0.3, seed=SEED),   # 대비 조정 추가
], name='augmentation_strong')

# 강한 증강 시각화
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
fig.suptitle('[도전 1] 강한 증강 (rotation=0.3)', fontsize=13)

for i, ax in enumerate(axes.flat):
    if i == 0:
        ax.imshow(sample_img.numpy())
        ax.set_title('원본', fontsize=10, color='red')
    else:
        aug_img = augmentation_strong(sample_batch, training=True)[0]
        ax.imshow(tf.clip_by_value(aug_img, 0.0, 1.0).numpy())
        ax.set_title(f'강한 증강 #{i}', fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------
# 도전 2: Z-score 정규화
#
# 수식: x' = (x - mu) / sigma
#
# CIFAR-10의 채널별 평균/표준편차 (사전 계산된 통계값)
# mean = [0.4914, 0.4822, 0.4465]  (R, G, B)
# std  = [0.2470, 0.2435, 0.2616]  (R, G, B)
# -----------------------------------------------------------

# 훈련 데이터에서 직접 계산
x_train_f = x_train.astype('float32') / 255.0
CIFAR_MEAN = x_train_f.mean(axis=(0, 1, 2))  # 채널별 평균
CIFAR_STD  = x_train_f.std(axis=(0, 1, 2))   # 채널별 표준편차

print('CIFAR-10 채널별 통계값:')
print(f'  Mean: R={CIFAR_MEAN[0]:.4f}, G={CIFAR_MEAN[1]:.4f}, B={CIFAR_MEAN[2]:.4f}')
print(f'  Std:  R={CIFAR_STD[0]:.4f},  G={CIFAR_STD[1]:.4f},  B={CIFAR_STD[2]:.4f}')

# Z-score 정규화 함수
MEAN_TENSOR = tf.constant(CIFAR_MEAN, dtype=tf.float32)  # (3,)
STD_TENSOR  = tf.constant(CIFAR_STD,  dtype=tf.float32)  # (3,)

def normalize_zscore(image, label):
    """Z-score 표준화: 채널별 평균 0, 표준편차 1로 변환"""
    image = tf.cast(image, tf.float32) / 255.0  # 먼저 [0,1]로
    image = (image - MEAN_TENSOR) / STD_TENSOR   # Z-score
    return image, label

# Z-score 파이프라인 구성
train_ds_zscore = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(normalize_zscore, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(BUFFER_SIZE, seed=SEED)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

for imgs_z, _ in train_ds_zscore.take(1):
    print(f'\n[도전 2] Z-score 정규화 결과:')
    print(f'  shape: {imgs_z.shape}')
    print(f'  R 채널 — 평균: {imgs_z[...,0].numpy().mean():.4f}, 표준편차: {imgs_z[...,0].numpy().std():.4f}')
    print(f'  G 채널 — 평균: {imgs_z[...,1].numpy().mean():.4f}, 표준편차: {imgs_z[...,1].numpy().std():.4f}')
    print(f'  B 채널 — 평균: {imgs_z[...,2].numpy().mean():.4f}, 표준편차: {imgs_z[...,2].numpy().std():.4f}')

In [ ]:
# -----------------------------------------------------------
# 도전 3: prefetch 유무에 따른 배치 로드 시간 측정
# -----------------------------------------------------------

NUM_EPOCHS = 3  # 여러 에포크에 걸쳐 측정 (캐시 효과 포함)

def time_pipeline(ds, epochs=NUM_EPOCHS, label=''):
    """dataset을 epochs 회 순회하는 시간(초)을 반환한다."""
    times = []
    for ep in range(epochs):
        t0 = time.perf_counter()
        batch_count = 0
        for _ in ds:
            batch_count += 1
        elapsed = time.perf_counter() - t0
        times.append(elapsed)
        print(f'  [{label}] 에포크 {ep+1}: {elapsed:.3f}초  ({batch_count}배치)')
    return times

# prefetch 없는 파이프라인
ds_no_prefetch = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(normalize_minmax, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(BUFFER_SIZE, seed=SEED)
    .batch(BATCH_SIZE)
    # .prefetch() 없음
)

# prefetch 있는 파이프라인
ds_with_prefetch = ds_no_prefetch.prefetch(AUTOTUNE)

print('=== prefetch 없음 ===')
times_no = time_pipeline(ds_no_prefetch, label='no prefetch')

print('\n=== prefetch 있음 (AUTOTUNE) ===')
times_yes = time_pipeline(ds_with_prefetch, label='with prefetch')

# 결과 요약
print('\n=== 결과 요약 ===')
print(f'prefetch 없음 — 평균: {np.mean(times_no):.3f}초')
print(f'prefetch 있음 — 평균: {np.mean(times_yes):.3f}초')
ratio = np.mean(times_no) / np.mean(times_yes)
print(f'속도 향상: {ratio:.2f}x  ({"빠름" if ratio > 1 else "차이 없음"})')

# 시각화
fig, ax = plt.subplots(figsize=(8, 4))
epochs_x = list(range(1, NUM_EPOCHS + 1))
ax.plot(epochs_x, times_no,  'o-', label='prefetch 없음', color='tomato')
ax.plot(epochs_x, times_yes, 's-', label='prefetch 있음 (AUTOTUNE)', color='steelblue')
ax.set_xlabel('에포크')
ax.set_ylabel('소요 시간 (초)')
ax.set_title('prefetch 유무에 따른 배치 로드 시간 비교')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 정리

### 이번 실습에서 구축한 파이프라인 전체 구조

```
CIFAR-10 numpy 배열
  │
  ├─ from_tensor_slices()
  │
  ├─ .map(normalize_minmax)   ← Min-Max 정규화: x' = x / 255
  │
  ├─ .cache()                 ← 메모리 캐시 (2번째 에포크부터 고속)
  │
  ├─ .shuffle(10000)          ← 랜덤 셔플 (훈련 시만)
  │
  ├─ .batch(32)               ← 배치 생성
  │
  ├─ .map(augment_fn)         ← RandomFlip + RandomRotation + RandomZoom
  │                              (훈련 시만, 배치 단위 처리)
  │
  └─ .prefetch(AUTOTUNE)      ← 비동기 프리패치
```

### 도전 과제 결과 요약

| 도전 | 핵심 포인트 |
|------|------------|
| 1. 강한 증강 (rotation=0.3) | 과도한 회전은 객체 인식을 어렵게 만들 수 있음 |
| 2. Z-score 정규화 | 채널별 평균 ≈ 0, 표준편차 ≈ 1; 심층 네트워크 학습 안정화 |
| 3. prefetch 성능 측정 | 특히 1번째 에포크(캐시 미스) 이후에 효과가 두드러짐 |